# Import Modules

In [10]:
import h2o
import pandas

In [11]:
project_path = "/gtc-2017"

# Connect or Start H2O

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 56 mins
H2O cluster version:,3.11.0.226
H2O cluster version age:,2 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,26.48 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


# Import Data

Import data into H2O.

In [13]:
mnist_training = h2o.import_file(project_path+"/data/mnist-training.csv")
mnist_testing = h2o.import_file(project_path+"/data/mnist-testing.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Change the "label" column to a factor (i.e. categorical feature).

In [14]:
mnist_training["label"] = mnist_training["label"].asfactor()
mnist_testing["label"] = mnist_testing["label"].asfactor()

# Explore Data

In [15]:
mnist_training.head()

uri,label
/gtc-2017/data/mnist_png/training/6/6453.png,6
/gtc-2017/data/mnist_png/training/6/13970.png,6
/gtc-2017/data/mnist_png/training/6/21967.png,6
/gtc-2017/data/mnist_png/training/6/11277.png,6
/gtc-2017/data/mnist_png/training/6/27350.png,6
/gtc-2017/data/mnist_png/training/6/32119.png,6
/gtc-2017/data/mnist_png/training/6/44201.png,6
/gtc-2017/data/mnist_png/training/6/58770.png,6
/gtc-2017/data/mnist_png/training/6/39440.png,6
/gtc-2017/data/mnist_png/training/6/41988.png,6


Get the number of output classes from the data.

In [16]:
num_classes = mnist_training["label"].nlevels()[0]
print(num_classes)

10


# Test GPU Driver

In [17]:
!nvidia-smi

Mon May  8 20:52:54 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0      On |                  N/A |
| 28%   38C    P8    11W / 180W |    796MiB /  8112MiB |     28%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Define Custom Network

In [18]:
import tensorflow as tf
import json
from keras.layers.core import Dense, Flatten, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
from keras.objectives import categorical_crossentropy
from tensorflow.python.framework import ops

def keras_model(w, h, channels, num_classes):
    graph = tf.Graph()
    with graph.as_default():
        size = w * h * channels
        
        inp = tf.placeholder(tf.float32, [None, size])
        
        labels = tf.placeholder(tf.float32, [None, num_classes])
        
        x = Reshape((w, h, channels))(inp)
        x = Conv2D(20, (5,5), padding="same", activation="relu")(x)
        x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(50, (5,5), padding="same", activation="relu")(x)
        x = MaxPooling2D((2,2))(x)
        
        x = Flatten()(x)
        
        x = Dense(num_classes)(x)
        
        out = Dense(num_classes)(x)
        
        predictions = tf.nn.softmax(out)
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=out))
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
        
        init_op = tf.global_variables_initializer()
        
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init_op.name)
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("logits", out)
        tf.add_to_collection("predictions", predictions)
        
        meta = json.dumps({
            "inputs": {"batch_image_input": inp.name, "categorical_labels": labels.name},
            "outputs": {"categorical_logits": out.name, "layers": ",".join([m.name for m in tf.get_default_graph().get_operations()])},
            "parameters": {}
        })
        
        tf.add_to_collection("meta", meta)
        
        saver = tf.train.Saver()
        filename= project_path+"/models/keras_tensorflow_lenet.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())
        return(filename)

Using TensorFlow backend.


In [19]:
network_def_path = keras_model(28, 28, 1, num_classes)

# Build Deep Water Keras (TensorFlow) Model

In [20]:
from h2o.estimators.deepwater import H2ODeepWaterEstimator
model_mnist_mylenet_keras = H2ODeepWaterEstimator(epochs=80,
                                            network_definition_file=network_def_path,
                                            backend="tensorflow",
                                            image_shape=[28,28],
                                            channels=1, model_id="model_mnist_mylenet_keras")
model_mnist_mylenet_keras.train(x=["uri"], y="label", training_frame=mnist_training, validation_frame=mnist_testing)

deepwater Model Build progress: |█████████████████████████████████████████| 100%


In [21]:
model_mnist_mylenet_keras.show()

Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  model_mnist_mylenet_keras


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.00607328251334
RMSE: 0.0779312678797
LogLoss: 0.0258161830826
Mean Per-Class Error: 0.00759141551164
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
972.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0030769,3 / 975
0.0,1145.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0052129,"6 / 1,151"
0.0,2.0,1006.0,4.0,0.0,0.0,1.0,1.0,8.0,0.0,0.0156556,"16 / 1,022"
0.0,0.0,3.0,1025.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0077444,"8 / 1,033"
0.0,1.0,0.0,0.0,958.0,0.0,2.0,2.0,0.0,6.0,0.0113519,11 / 969
0.0,0.0,0.0,5.0,0.0,839.0,3.0,0.0,1.0,2.0,0.0129412,11 / 850
0.0,0.0,0.0,0.0,0.0,0.0,992.0,0.0,1.0,0.0,0.0010070,1 / 993
0.0,1.0,1.0,1.0,0.0,0.0,0.0,1055.0,0.0,1.0,0.0037771,"4 / 1,059"
1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,936.0,1.0,0.0031949,3 / 939
0.0,1.0,0.0,0.0,7.0,0.0,0.0,4.0,0.0,992.0,0.0119522,"12 / 1,004"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9924962
2,0.9989995
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0



ModelMetricsMultinomial: deepwater
** Reported on validation data. **

MSE: 0.0117393816579
RMSE: 0.108348427113
LogLoss: 0.0580633801598
Mean Per-Class Error: 0.0136599369188
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
968.0,0.0,1.0,0.0,0.0,1.0,6.0,1.0,3.0,0.0,0.0122449,12 / 980
1.0,1123.0,1.0,1.0,0.0,1.0,5.0,1.0,2.0,0.0,0.0105727,"12 / 1,135"
6.0,2.0,1012.0,5.0,0.0,0.0,2.0,4.0,1.0,0.0,0.0193798,"20 / 1,032"
0.0,0.0,0.0,1002.0,0.0,2.0,0.0,2.0,4.0,0.0,0.0079208,"8 / 1,010"
0.0,1.0,0.0,0.0,964.0,0.0,4.0,3.0,2.0,8.0,0.0183299,18 / 982
1.0,1.0,0.0,6.0,0.0,874.0,3.0,0.0,5.0,2.0,0.0201794,18 / 892
2.0,1.0,0.0,0.0,0.0,0.0,952.0,0.0,3.0,0.0,0.0062630,6 / 958
0.0,0.0,7.0,0.0,0.0,0.0,0.0,1017.0,1.0,3.0,0.0107004,"11 / 1,028"
2.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,966.0,2.0,0.0082136,8 / 974
1.0,4.0,0.0,5.0,7.0,3.0,0.0,3.0,0.0,986.0,0.0227948,"23 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9864
2,0.9965
3,0.9994
4,0.9998
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2017-05-08 20:52:55,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2017-05-08 20:52:56,1.256 sec,8533 obs/sec,0.0170667,1,1024.0,0.5208416,1.5063509,0.3028514,0.5236829,1.5059840,0.3088
,2017-05-08 20:53:06,11.631 sec,16298 obs/sec,2.6624,156,159744.0,0.1070740,0.0497958,0.0143072,0.1310304,0.0761871,0.0211
,2017-05-08 20:53:13,18.619 sec,15969 obs/sec,4.2837333,251,257024.0,0.1051433,0.0452353,0.0139070,0.1229485,0.0701109,0.0186
,2017-05-08 20:53:20,25.554 sec,15192 obs/sec,5.6661333,332,339968.0,0.0915219,0.0348545,0.0104052,0.1186294,0.0660772,0.0171
,2017-05-08 20:53:27,32.173 sec,14795 obs/sec,6.9802667,409,418816.0,0.0779313,0.0258162,0.0075038,0.1083484,0.0580634,0.0136
,2017-05-08 20:53:34,39.234 sec,14380 obs/sec,8.2944,486,497664.0,0.0849103,0.0328230,0.0088044,0.1120859,0.0701819,0.0147
,2017-05-08 20:53:42,46.910 sec,13986 obs/sec,9.6768,567,580608.0,0.0845837,0.0349662,0.0087044,0.1156877,0.0840219,0.0151
,2017-05-08 20:53:49,54.660 sec,13740 obs/sec,11.1104,651,666624.0,0.0649014,0.0193741,0.0053027,0.1103369,0.0712966,0.0147
,2017-05-08 20:53:57,1 min 2.148 sec,13558 obs/sec,12.4928,732,749568.0,0.0635988,0.0189405,0.0047024,0.1131730,0.0922341,0.014
